In [2]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
import pyximport 
pyximport.install()
import random
import numpy as np
from data_utils import load_CIFAR10
import matplotlib.pyplot as plt
from scipy import flip
from Layers import *
from Layers2 import*
from Network import *
random.seed(7)

In [3]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'



In [4]:
# Load the raw CIFAR-10 data.
cifar10_dir = 'cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

In [5]:
# As a sanity check, we print out the size of the training and test data.
print ('Training data shape: ', X_train.shape)
print ('Training labels shape: ', y_train.shape)
print ('Test data shape: ', X_test.shape)
print ('Test labels shape: ', y_test.shape)

('Training data shape: ', (50000, 32, 32, 3))
('Training labels shape: ', (50000,))
('Test data shape: ', (10000, 32, 32, 3))
('Test labels shape: ', (10000,))


In [6]:
# preprocessing
# zero centering 
#meanimage=np.empty(0,int)
meanimage=np.mean(X_train, axis=0)
stdimage= np.std(X_train, axis=0)
X_train -= meanimage
X_train =X_train/stdimage
X_test -= meanimage
X_test = X_test/stdimage

In [7]:
#augmentation
X_train_flip= flip(X_train, 2)
X_train= np.concatenate((X_train, X_train_flip), axis=0)
y_train= np.concatenate((y_train, y_train), axis=0)
print (X_train.shape)

(100000, 32, 32, 3)


In [8]:
X_train = np.transpose(X_train, axes=(0, 3, 1, 2))
X_test = np.transpose(X_test, axes=(0, 3, 1, 2))
print (X_train.shape)

(100000, 3, 32, 32)


In [9]:
#X_train = np.reshape(X_train, (X_train.shape[0], -1))
#X_test = np.reshape(X_test, (X_test.shape[0], -1))

In [10]:
#layers=[1000, 500, 200]
#net=FCN(layers)
#net.train(X_train[0:70000],y_train[0:70000],X_train[70000:100000],y_train[70000:100000])
net=network()
net.add(conv(filters=16, channels=3, width=32, height=32))
net.add(relu())

net.add(conv(filters=32, channels=16, width=32, height=32))
net.add(relu())
net.add(maxpool())

net.add(conv(filters=64, channels=32, width=16, height=16))
net.add(relu())

net.add(conv(filters=64, channels=64, width=16, height=16))
net.add(relu())
net.add(maxpool())

net.add(flatten())
net.add(layer(4096,1000))
net.add(relu())
net.add(layer(1000,500))
net.add(relu())
net.add(layer(500,200))
net.add(relu())
net.add(layer(200,10))
net.add(lossfunc())
net.train(X_train[:70000],y_train[:70000],X_train[70000:],y_train[70000:],batch_size=200)
net.evaluate(X_test,y_test)

Epoch: 0 tr_loss = 1.291902 val_loss = 0.938153 tr_acc = 0.532814 val_acc = 0.672067

Epoch: 1 tr_loss = 0.801542 val_loss = 0.739852 tr_acc = 0.717529 val_acc = 0.741700

Epoch: 2 tr_loss = 0.584764 val_loss = 0.743765 tr_acc = 0.796271 val_acc = 0.746033

Epoch: 3 tr_loss = 0.439947 val_loss = 0.812740 tr_acc = 0.845914 val_acc = 0.747000

Epoch: 4 tr_loss = 0.320257 val_loss = 0.827137 tr_acc = 0.887686 val_acc = 0.761900

Epoch: 5 tr_loss = 0.232055 val_loss = 0.953784 tr_acc = 0.918214 val_acc = 0.758633

Epoch: 6 tr_loss = 0.176774 val_loss = 0.963667 tr_acc = 0.938771 val_acc = 0.762033

Epoch: 7 tr_loss = 0.118619 val_loss = 1.164159 tr_acc = 0.959386 val_acc = 0.754533

Epoch: 8 tr_loss = 0.083700 val_loss = 1.224458 tr_acc = 0.971529 val_acc = 0.764000

Epoch: 9 tr_loss = 0.064884 val_loss = 1.331687 tr_acc = 0.978543 val_acc = 0.761600

Epoch: 10 tr_loss = 0.060377 val_loss = 1.184813 tr_acc = 0.980729 val_acc = 0.775733

Epoch: 11 tr_loss = 0.048670 val_loss = 1.384061 tr_a

KeyboardInterrupt: 

In [19]:
c=0
for i in range (net.num):
    if (net.layers[i].layername()=="conv" or net.layers[i].layername()=="layer") :
        #net.layers[i].setparams(net.bestweights[c])
        net.layers[i].w, net.layers[i].b = net.bestweights[c]
        c=c+1

In [21]:
test_loss,test_acc=net.evaluate(X_test, y_test, step_size=200)
print ('Test set accuracy: ', test_acc)

('Test set accuracy: ', 0.75960000000000005)


In [20]:
ccrn=np.zeros(10)
for j in range (10):
    I = [i for i , y in enumerate(y_test) if y == j]
    _, ccrn[j] = net.evaluate(X_test[I], y_test[I], step_size = 250)
    print("CCrn for class %d is %f" % (j, ccrn[j]))

CCrn for class 0 is 0.825000
CCrn for class 1 is 0.865000
CCrn for class 2 is 0.664000
CCrn for class 3 is 0.520000
CCrn for class 4 is 0.722000
CCrn for class 5 is 0.672000
CCrn for class 6 is 0.829000
CCrn for class 7 is 0.811000
CCrn for class 8 is 0.871000
CCrn for class 9 is 0.817000
